<a href="https://colab.research.google.com/github/rajrajeshwarpandya/dashlab-project/blob/main/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile naive.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

__global__ void mat_mul_naive(const float *a, const float *b, float *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < ms && col < ms) {
        float sum = 0.0f;
        for (int k = 0; k < ms; ++k)
            sum += a[row * ms + k] * b[k * ms + col];
        c[row * ms + col] = sum;
    }
}

void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));

    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block(bsize, bsize);
    dim3 grid((ms + bsize - 1) / bsize, (ms + bsize - 1) / bsize);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iterations = 100;
    float total_ms = 0.0f;

    cudaMemset(d_c, 0, bytes);
    mat_mul_naive<<<grid, block>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    total_ms = 0.0f;
    for (int i = 0; i < iterations; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        mat_mul_naive<<<grid, block>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    double avg_ms = (double)total_ms / iterations;
    double gflops = (2.0 * ms * ms * ms) / (avg_ms / 1e3) / 1e9;
    printf("\n1. Naive Kernel(100):\n", iterations);
    printf("   Time: %.3f ms\n", avg_ms);
    printf("   Performance: %.3f GFLOPS\n", gflops);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing naive.cu


In [2]:
!nvcc -arch=sm_75 naive.cu -o naive.cu
!./naive.cu

naive.cu(74): warning #225-D: the format string ends before this argument
      printf("\n1. Naive Kernel(100):\n", iterations);
                                          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

naive.cu: In function ‘int main()’:
naive.cu:74:8: warning: too many arguments for format []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-extra-args-Wformat-extra-args]8;;]
   74 |     printf("\n1. Naive Kernel(100):\n", iterations);
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~

1. Naive Kernel(100):
   Time: 4.545 ms
   Performance: 472.454 GFLOPS


In [3]:
%%writefile tiled.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

__global__ void mat_mul_tiled(const float *a, const float *b, float *c) {
    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;
    int i = by * blockDim.y + ty;
    int j = bx * blockDim.x + tx;

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    float sum = 0.0f;
    int num_tiles = (ms + bsize - 1) / bsize;

    for (int t = 0; t < num_tiles; t++) {
        int a_col = t * bsize + tx;
        int b_row = t * bsize + ty;

        if (i < ms && a_col < ms)
            as[ty][tx] = a[i * ms + a_col];
        else
            as[ty][tx] = 0.0f;

        if (b_row < ms && j < ms)
            bs[ty][tx] = b[b_row * ms + j];
        else
            bs[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < bsize; k++)
            sum += as[ty][k] * bs[k][tx];

        __syncthreads();
    }

    if (i < ms && j < ms)
        c[i * ms + j] = sum;
}

void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));

    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block(bsize, bsize);
    dim3 grid((ms + bsize - 1) / bsize, (ms + bsize - 1) / bsize);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iterations = 100;
    float total_ms = 0.0f;

    cudaMemset(d_c, 0, bytes);
    mat_mul_tiled<<<grid, block>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    total_ms = 0.0f;
    for (int i = 0; i < iterations; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        mat_mul_tiled<<<grid, block>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    double avg_ms = (double)total_ms / iterations;
    double gflops = (2.0 * ms * ms * ms) / (avg_ms / 1e3) / 1e9;
    printf("\n2. Tiled Kernel (100):\n", bsize, iterations);
    printf("   Time: %.3f ms\n", avg_ms);
    printf("   Performance: %.3f GFLOPS\n", gflops);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing tiled.cu


In [4]:
!nvcc -arch=sm_75 tiled.cu -o tiled.cu
!./tiled.cu

tiled.cu(102): warning #225-D: the format string ends before this argument
      printf("\n2. Tiled Kernel (100):\n", 32, iterations);
                                           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

tiled.cu: In function ‘int main()’:
tiled.cu:102:8: warning: too many arguments for format []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-extra-args-Wformat-extra-args]8;;]
  102 |     printf("\n2. Tiled Kernel (100):\n", bsize, iterations);
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~

2. Tiled Kernel (100):
   Time: 3.816 ms
   Performance: 562.728 GFLOPS


In [17]:
%%writefile registered.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

#define reg_dim 4
#define block_x 8
#define block_y 8

__global__ void mat_mul_reg_tiled(const float *a, const float *b, float *c) {

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int i = (by * block_y + ty) * reg_dim;
    int j = (bx * block_x + tx) * reg_dim;

    float c_sub[reg_dim][reg_dim] = {{0.0f}};

    int num_tiles = (ms + bsize - 1) / bsize;

    for (int t = 0; t < num_tiles; t++) {

        for(int row = 0; row < reg_dim; row++) {
            for(int col = 0; col < reg_dim; col++) {

                int a_row = i + row;
                int a_col = t * bsize + tx * reg_dim + col;
                int as_row = ty * reg_dim + row;
                int as_col = tx * reg_dim + col;

                if (a_row < ms && a_col < ms)
                    as[as_row][as_col] = a[a_row * ms + a_col];
                else
                    as[as_row][as_col] = 0.0f;

                int b_row = t * bsize + ty * reg_dim + row;
                int b_col = j + col;
                int bs_row = ty * reg_dim + row;
                int bs_col = tx * reg_dim + col;

                if (b_row < ms && b_col < ms)
                    bs[bs_row][bs_col] = b[b_row * ms + b_col];
                else
                    bs[bs_row][bs_col] = 0.0f;
            }
        }
        __syncthreads();

        for (int k = 0; k < bsize; k++) {

            float a_reg[reg_dim];
            float b_reg[reg_dim];

            for (int m = 0; m < reg_dim; m++)
                a_reg[m] = as[ty * reg_dim + m][k];

            for (int n = 0; n < reg_dim; n++)
                b_reg[n] = bs[k][tx * reg_dim + n];

            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    c_sub[m][n] += a_reg[m] * b_reg[n];
                }
            }
        }
        __syncthreads();
    }

    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            if ( (i + m) < ms && (j + n) < ms)
                c[(i + m) * ms + (j + n)] = c_sub[m][n];
        }
    }
}


void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));

    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block_reg(block_x, block_y);
    dim3 grid_reg(ms / (block_x * reg_dim), ms / (block_y * reg_dim));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iterations = 100;
    float total_ms = 0.0f;

    cudaMemset(d_c, 0, bytes);
    mat_mul_reg_tiled<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    total_ms = 0.0f;
    for (int i = 0; i < iterations; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        mat_mul_reg_tiled<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
     #include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

#define reg_dim 4
#define block_x 8
#define block_y 8

__global__ void mat_mul_reg_tiled(const float *a, const float *b, float *c) {

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int i = (by * block_y + ty) * reg_dim;
    int j = (bx * block_x + tx) * reg_dim;

    float c_sub[reg_dim][reg_dim] = {{0.0f}};

    int num_tiles = (ms + bsize - 1) / bsize;

    for (int t = 0; t < num_tiles; t++) {

        for(int row = 0; row < reg_dim; row++) {
            for(int col = 0; col < reg_dim; col++) {

                int a_row = i + row;
                int a_col = t * bsize + tx * reg_dim + col;
                int as_row = ty * reg_dim + row;
                int as_col = tx * reg_dim + col;

                if (a_row < ms && a_col < ms)
                    as[as_row][as_col] = a[a_row * ms + a_col];
                else
                    as[as_row][as_col] = 0.0f;

                int b_row = t * bsize + ty * reg_dim + row;
                int b_col = j + col;
                int bs_row = ty * reg_dim + row;
                int bs_col = tx * reg_dim + col;

                if (b_row < ms && b_col < ms)
                    bs[bs_row][bs_col] = b[b_row * ms + b_col];
                else
                    bs[bs_row][bs_col] = 0.0f;
            }
        }
        __syncthreads();

        for (int k = 0; k < bsize; k++) {

            float a_reg[reg_dim];
            float b_reg[reg_dim];

            for (int m = 0; m < reg_dim; m++)
                a_reg[m] = as[ty * reg_dim + m][k];

            for (int n = 0; n < reg_dim; n++)
                b_reg[n] = bs[k][tx * reg_dim + n];

            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    c_sub[m][n] += a_reg[m] * b_reg[n];
                }
            }
        }
        __syncthreads();
    }

    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            if ( (i + m) < ms && (j + n) < ms)
                c[(i + m) * ms + (j + n)] = c_sub[m][n];
        }
    }
}


void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));

    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    i#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

#define reg_dim 4
#define block_x 8
#define block_y 8

__global__ void mat_mul_reg_tiled(const float *a, const float *b, float *c) {

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int i = (by * block_y + ty) * reg_dim;
    int j = (bx * block_x + tx) * reg_dim;

    float c_sub[reg_dim][reg_dim] = {{0.0f}};

    int num_tiles = (ms + bsize - 1) / bsize;

    for (int t = 0; t < num_tiles; t++) {

        for(int row = 0; row < reg_dim; row++) {
            for(int col = 0; col < reg_dim; col++) {

                int a_row = i + row;
                int a_col = t * bsize + tx * reg_dim + col;
                int as_row = ty * reg_dim + row;
                int as_col = tx * reg_dim + col;

                if (a_row < ms && a_col < ms)
                    as[as_row][as_col] = a[a_row * ms + a_col];
                else
                    as[as_row][as_col] = 0.0f;

                int b_row = t * bsize + ty * reg_dim + row;
                int b_col = j + col;
                int bs_row = ty * reg_dim + row;
                int bs_col = tx * reg_dim + col;

                if (b_row < ms && b_col < ms)
                    bs[bs_row][bs_col] = b[b_row * ms + b_col];
                else
                    bs[bs_row][bs_col] = 0.0f;
            }
        }
        __syncthreads();

        for (int k = 0; k < bsize; k++) {

            float a_reg[reg_dim];
            float b_reg[reg_dim];

            for (int m = 0; m < reg_dim; m++)
                a_reg[m] = as[ty * reg_dim + m][k];

            for (int n = 0; n < reg_dim; n++)
                b_reg[n] = bs[k][tx * reg_dim + n];

            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    c_sub[m][n] += a_reg[m] * b_reg[n];
                }
            }
        }
        __syncthreads();
    }

    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            if ( (i + m) < ms && (j + n) < ms)
                c[(i + m) * ms + (j + n)] = c_sub[m][n];
        }
    }
}


void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));

    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block_reg(block_x, block_y);
    dim3 grid_reg(ms / (block_x * reg_dim), ms / (block_y * reg_dim));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iterations = 100;
    float total_ms = 0.0f;

    cudaMemset(d_c, 0, bytes);
    mat_mul_reg_tiled<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    total_ms = 0.0f;
    for (int i = 0; i < iterations; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        mat_mul_reg_tiled<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    double avg_ms = (double)total_ms / iterations;
    double gflops = (2.0 * ms * ms * ms) / (avg_ms / 1e3) / 1e9;
    printf("\n3. Register Tiled Kernel (100):\n",
           block_x, block_y, reg_dim, reg_dim, iterations);
    printf("   Time: %.3f ms\n", avg_ms);
    printf("   Performance: %.3f GFLOPS\n", gflops);


    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}it_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block_reg(block_x, block_y);
    dim3 grid_reg(ms / (block_x * reg_dim), ms / (block_y * reg_dim));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iterations = 100;
    float total_ms = 0.0f;

    cudaMemset(d_c, 0, bytes);
    mat_mul_reg_tiled<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    total_ms = 0.0f;
    for (int i = 0; i < iterations; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        mat_mul_reg_tiled<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    double avg_ms = (double)total_ms / iterations;
    double gflops = (2.0 * ms * ms * ms) / (avg_ms / 1e3) / 1e9;
    printf("\n3. Register Tiled Kernel (100):\n",
           block_x, block_y, reg_dim, reg_dim, iterations);
    printf("   Time: %.3f ms\n", avg_ms);
    printf("   Performance: %.3f GFLOPS\n", gflops);


    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}   total_ms += ms_kernel;
    }
    double avg_ms = (double)total_ms / iterations;
    double gflops = (2.0 * ms * ms * ms) / (avg_ms / 1e3) / 1e9;
    printf("\n3. Register Tiled Kernel (100):\n",
           block_x, block_y, reg_dim, reg_dim, iterations);
    printf("   Time: %.3f ms\n", avg_ms);
    printf("   Performance: %.3f GFLOPS\n", gflops);


    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Overwriting registered.cu


In [6]:
!nvcc -arch=sm_75 registered.cu -o registered.cu
!./registered.cu

registered.cu(149): error: expected a ";"
  __attribute__((global)) void mat_mul_reg_tiled(const float *a, const float *b, float *c) {
                                                                                           ^

registered.cu(171): warning #12-D: parsing restarts here after previous syntax error
                  int a_row = i + row;
                                     ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

registered.cu(172): error: identifier "t" is undefined
                  int a_col = t * 32 + tx * 4 + col;
                              ^

registered.cu(172): error: identifier "tx" is undefined
                  int a_col = t * 32 + tx * 4 + col;
                                       ^

registered.cu(172): error: identifier "col" is undefined
                  int a_col = t * 32 + tx * 4 + col;
                                                ^

registered.cu(173): error: identifier "ty" is undefined
                  i

In [7]:
%%writefile db.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>

#define tile_dim 32
#define n 1024
#define warmup_runs 10
#define benchmark_runs 100

__global__ void matMulDoubleBuffered(const float *a, const float *b, float *c, int m)
{
    __shared__ float sh_a[2][tile_dim][tile_dim];
    __shared__ float sh_b[2][tile_dim][tile_dim];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;

    int c_row = by * tile_dim + ty;
    int c_col = bx * tile_dim + tx;

    float acc = 0.0f;
    int num_tiles = m / tile_dim;

    if (c_row >= m || c_col >= m) return;

    sh_a[0][ty][tx] = a[c_row * m + tx];
    sh_b[0][ty][tx] = b[ty * m + c_col];
    __syncthreads();

    for (int k = 0; k < num_tiles - 1; ++k)
    {
        int current_buf = k % 2;
        int next_buf = (k + 1) % 2;

        sh_a[next_buf][ty][tx] = a[c_row * m + (k + 1) * tile_dim + tx];
        sh_b[next_buf][ty][tx] = b[((k + 1) * tile_dim + ty) * m + c_col];

        for (int i = 0; i < tile_dim; ++i)
        {
            acc += sh_a[current_buf][ty][i] * sh_b[current_buf][i][tx];
        }
        __syncthreads();
    }

    int last_buf = (num_tiles - 1) % 2;
    for (int i = 0; i < tile_dim; ++i)
    {
        acc += sh_a[last_buf][ty][i] * sh_b[last_buf][i][tx];
    }

    c[c_row * m + c_col] = acc;
}

void init_matrix(float *m, int size)
{
    for (int i = 0; i < size; ++i)
    {
        m[i] = (float)rand() / (float)RAND_MAX;
    }
}

int main(void)
{

    size_t bytes = (size_t)n * n * sizeof(float);

    float *h_a = (float *)malloc(bytes);
    float *h_b = (float *)malloc(bytes);

    srand(time(NULL));
    init_matrix(h_a, n * n);
    init_matrix(h_b, n * n);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);

    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(tile_dim, tile_dim);
    dim3 blocksPerGrid(n / tile_dim, n / tile_dim);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    for (int i = 0; i < warmup_runs; ++i)
    {
        matMulDoubleBuffered<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);
    }
    cudaDeviceSynchronize();

    float total_time_ms = 0.0f;
    for (int i = 0; i < benchmark_runs; ++i)
    {
        cudaEventRecord(start);
        matMulDoubleBuffered<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float elapsed_ms;
        cudaEventElapsedTime(&elapsed_ms, start, stop);
        total_time_ms += elapsed_ms;
    }

    cudaDeviceSynchronize();

    double avg_time_ms = total_time_ms / benchmark_runs;
    double avg_time_s = avg_time_ms / 1000.0;

    double flops = 2.0 * (double)n * (double)n * (double)n;
    double gflops = (flops / avg_time_s) / 1e9;
    printf("Avg Time: %.4f ms\n", avg_time_ms);
    printf("GFLOPS: %.4f\n", gflops);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    free(h_a);
    free(h_b);

    cudaDeviceReset();

    return 0;
}

Writing db.cu


In [8]:
!nvcc -arch=sm_75 db.cu -o db.cu
!./db.cu

Avg Time: 3.4434 ms
GFLOPS: 623.6585


In [9]:
%%writefile cr.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

#define reg_dim 4
#define block_x 8
#define block_y 8

__global__ void mat_mul_reg_coalesced(const float *a, const float *b, float *c) {

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    int ty = threadIdx.y;
    int tx = threadIdx.x;
    int tid = ty * block_x + tx;
    int ib = blockIdx.y * bsize;
    int jb = blockIdx.x * bsize;

    float csub[reg_dim][reg_dim] = {{0.0f}};
    int num_tiles = ms / bsize;

    for (int t = 0; t < num_tiles; t++) {

        int ga_col = t * bsize;
        int gb_row = t * bsize;
        for (int k = 0; k < 16; ++k) {
            int idx = tid + k * 64;
            int sr = idx / 32;
            int sc = idx % 32;
            as[sr][sc] = a[(ib + sr) * ms + (ga_col + sc)];
            bs[sr][sc] = b[(gb_row + sr) * ms + (jb + sc)];
        }
        __syncthreads();

        for (int k = 0; k < bsize; k++) {
            float areg[reg_dim];
            float breg[reg_dim];
            for (int m = 0; m < reg_dim; m++) areg[m] = as[ty * reg_dim + m][k];
            for (int n = 0; n < reg_dim; n++) breg[n] = bs[k][tx * reg_dim + n];
            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    csub[m][n] += areg[m] * breg[n];
                }
            }
        }
        __syncthreads();
    }

    __shared__ float cs[bsize][bsize];
    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            cs[ty * reg_dim + m][tx * reg_dim + n] = csub[m][n];
        }
    }
    __syncthreads();

    for (int k = 0; k < 16; ++k) {
        int idx = tid + k * 64;
        int sr = idx / 32;
        int sc = idx % 32;
        c[(ib + sr) * ms + (jb + sc)] = cs[sr][sc];
    }
}

void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));
    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block_reg(block_x, block_y);
    dim3 grid_reg(ms / bsize, ms / bsize);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iters = 100;
    float total_ms = 0.0f;
    double gflops_base = (2.0 * ms * ms * ms) / 1e9;

    mat_mul_reg_coalesced<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
    cudaMemset(d_c, 0, bytes);
    total_ms = 0.0f;
    for (int i = 0; i < iters; ++i) {
        cudaEventRecord(start);
        mat_mul_reg_coalesced<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    printf("   Time: %.3f ms\n", (double)total_ms / iters);
    printf   ("   Performance: %.3f GFLOPS\n", gflops_base / ((double)total_ms / iters / 1e3));

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing cr.cu


In [10]:
!nvcc -arch=sm_75 cr.cu -o cr.cu
!./cr.cu

   Time: 1.703 ms
   Performance: 1261.199 GFLOPS


In [11]:
%%writefile nc.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>
#include <time.h>

#define ms 1024
#define bsize 32

#define reg_dim 4
#define block_x 8
#define block_y 8

__global__ void mat_mul_reg_buffered(const float *a, const float *b, float *c) {

    __shared__ float as[2][bsize][bsize];
    __shared__ float bs[2][bsize][bsize];

    int ty = threadIdx.y;
    int tx = threadIdx.x;
    int tid = ty * block_x + tx;
    int ib = blockIdx.y * bsize;
    int jb = blockIdx.x * bsize;

    float csub[reg_dim][reg_dim] = {{0.0f}};
    int num_tiles = ms / bsize;

    int ga_col = 0 * bsize;
    int gb_row = 0 * bsize;
    for (int k = 0; k < 16; ++k) {
        int idx = tid + k * 64;
        int sr = idx / 32;
        int sc = idx % 32;
        as[0][sr][sc] = a[(ib + sr) * ms + (ga_col + sc)];
        bs[0][sr][sc] = b[(gb_row + sr) * ms + (jb + sc)];
    }
    __syncthreads();

    for (int t = 1; t < num_tiles; t++) {

        int comp_buf = (t - 1) % 2;
        int load_buf = t % 2;

        ga_col = t * bsize;
        gb_row = t * bsize;
        for (int k = 0; k < 16; ++k) {
            int idx = tid + k * 64;
            int sr = idx / 32;
            int sc = idx % 32;
            as[load_buf][sr][sc] = a[(ib + sr) * ms + (ga_col + sc)];
            bs[load_buf][sr][sc] = b[(gb_row + sr) * ms + (jb + sc)];
        }

        for (int k = 0; k < bsize; k++) {
            float areg[reg_dim];
            float breg[reg_dim];
            for (int m = 0; m < reg_dim; m++) areg[m] = as[comp_buf][ty * reg_dim + m][k];
            for (int n = 0; n < reg_dim; n++) breg[n] = bs[comp_buf][k][tx * reg_dim + n];
            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    csub[m][n] += areg[m] * breg[n];
                }
            }
        }

        __syncthreads();
    }

    int last_buf = (num_tiles - 1) % 2;
    for (int k = 0; k < bsize; k++) {
        float areg[reg_dim];
        float breg[reg_dim];
        for (int m = 0; m < reg_dim; m++) areg[m] = as[last_buf][ty * reg_dim + m][k];
        for (int n = 0; n < reg_dim; n++) breg[n] = bs[last_buf][k][tx * reg_dim + n];
        for (int m = 0; m < reg_dim; m++) {
            for (int n = 0; n < reg_dim; n++) {
                csub[m][n] += areg[m] * breg[n];
            }
        }
    }

    __shared__ float cs[bsize][bsize];
    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            cs[ty * reg_dim + m][tx * reg_dim + n] = csub[m][n];
        }
    }
    __syncthreads();

    for (int k = 0; k < 16; ++k) {
        int idx = tid + k * 64;
        int sr = idx / 32;
        int sc = idx % 32;
        c[(ib + sr) * ms + (jb + sc)] = cs[sr][sc];
    }
}


void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));
    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    dim3 block_reg(block_x, block_y);
    dim3 grid_reg(ms / bsize, ms / bsize);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int iters = 100;
    float total_ms = 0.0f;
    double gflops_base = (2.0 * ms * ms * ms) / 1e9;

    mat_mul_reg_buffered<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
    cudaMemset(d_c, 0, bytes);
    total_ms = 0.0f;
    for (int i = 0; i < iters; ++i) {
        cudaEventRecord(start);
        mat_mul_reg_buffered<<<grid_reg, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    printf("   Time: %.3f ms\n", (double)total_ms / iters);
    printf   ("   Performance: %.3f GFLOPS\n", gflops_base / ((double)total_ms / iters / 1e3));

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing nc.cu


In [12]:
!nvcc -arch=sm_75 nc.cu -o nc.cu
!./nc.cu

   Time: 1.091 ms
   Performance: 1968.641 GFLOPS


In [13]:
%%writefile cubla.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <math.h>
#include <time.h>

#define ms 1024

void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

int main() {
    srand((unsigned int)time(NULL));
    size_t bytes = ms * ms * sizeof(float);

    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cublasHandle_t handle;
    cublasCreate(&handle);

    const float alpha = 1.0f;
    const float beta = 0.0f;

    int iters = 100;
    float total_ms = 0.0f;
    double gflops_base = (2.0 * ms * ms * ms) / 1e9;

    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, ms, ms, ms, &alpha, d_b, ms, d_a, ms, &beta, d_c, ms);

    cudaMemset(d_c, 0, bytes);
    total_ms = 0.0f;
    for (int i = 0; i < iters; ++i) {
        cudaEventRecord(start);
        cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, ms, ms, ms, &alpha, d_b, ms, d_a, ms, &beta, d_c, ms);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        float ms_kernel = 0.0f;
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }

    printf("\n6. cuBLAS Kernel (Avg over %d runs):\n", iters);
    printf("   Time: %.3f ms\n", (double)total_ms / iters);
    printf("   Performance: %.3f GFLOPS\n", gflops_base / ((double)total_ms / iters / 1e3));

    cublasDestroy(handle);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing cubla.cu


In [14]:
!nvcc -o cubla cubla.cu -lcublas
!./cubla


6. cuBLAS Kernel (Avg over 100 runs):
   Time: 0.831 ms
   Performance: 2584.770 GFLOPS


In [15]:
%%writefile bm.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <math.h>

// --- Common Definitions ---
#define ms 1024         // Matrix Size (ms x ms)
#define bsize 32        // Block Size (bsize x bsize)

// --- Definitions for Register Tiling Kernels ---
#define reg_dim 4       // Register tile dimensions (reg_dim x reg_dim)
#define block_x 8       // Threads in x-dim for register kernels
#define block_y 8       // Threads in y-dim for register kernels (8x8 = 64 threads)

// --- Benchmarking Runs ---
#define warmup_runs 10
#define benchmark_runs 100

// ===================================================================
// KERNEL 1: Naive (from naive.cu)
// ===================================================================
__global__ void kernel_naive(const float *a, const float *b, float *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < ms && col < ms) {
        float sum = 0.0f;
        for (int k = 0; k < ms; ++k)
            sum += a[row * ms + k] * b[k * ms + col];
        c[row * ms + col] = sum;
    }
}

// ===================================================================
// KERNEL 2: Tiled (from tiled.cu)
// ===================================================================
__global__ void kernel_tiled(const float *a, const float *b, float *c) {
    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;
    int i = by * blockDim.y + ty;
    int j = bx * blockDim.x + tx;

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    float sum = 0.0f;
    int num_tiles = (ms + bsize - 1) / bsize;

    for (int t = 0; t < num_tiles; t++) {
        int a_col = t * bsize + tx;
        int b_row = t * bsize + ty;

        if (i < ms && a_col < ms)
            as[ty][tx] = a[i * ms + a_col];
        else
            as[ty][tx] = 0.0f;

        if (b_row < ms && j < ms)
            bs[ty][tx] = b[b_row * ms + j];
        else
            bs[ty][tx] = 0.0f;

        __syncthreads();

        for (int k = 0; k < bsize; k++)
            sum += as[ty][k] * bs[k][tx];

        __syncthreads();
    }

    if (i < ms && j < ms)
        c[i * ms + j] = sum;
}

// ===================================================================
// KERNEL 3: Tiled + Double Buffering (from double_buffering.cu)
// ===================================================================
__global__ void kernel_tiled_double_buffered(const float *a, const float *b, float *c, int m)
{
    __shared__ float sh_a[2][bsize][bsize];
    __shared__ float sh_b[2][bsize][bsize];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;

    int c_row = by * bsize + ty;
    int c_col = bx * bsize + tx;

    float acc = 0.0f;
    int num_tiles = m / bsize;

    if (c_row >= m || c_col >= m) return;

    sh_a[0][ty][tx] = a[c_row * m + tx];
    sh_b[0][ty][tx] = b[ty * m + c_col];
    __syncthreads();

    for (int k = 0; k < num_tiles - 1; ++k)
    {
        int current_buf = k % 2;
        int next_buf = (k + 1) % 2;

        sh_a[next_buf][ty][tx] = a[c_row * m + (k + 1) * bsize + tx];
        sh_b[next_buf][ty][tx] = b[((k + 1) * bsize + ty) * m + c_col];

        for (int i = 0; i < bsize; ++i)
        {
            acc += sh_a[current_buf][ty][i] * sh_b[current_buf][i][tx];
        }
        __syncthreads();
    }

    int last_buf = (num_tiles - 1) % 2;
    for (int i = 0; i < bsize; ++i)
    {
        acc += sh_a[last_buf][ty][i] * sh_b[last_buf][i][tx];
    }

    c[c_row * m + c_col] = acc;
}

// ===================================================================
// KERNEL 4: Register Tiled (Uncoalesced) (from registered.cu)
// ===================================================================
__global__ void kernel_reg_tiled(const float *a, const float *b, float *c) {

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    int by = blockIdx.y;
    int bx = blockIdx.x;
    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int i = (by * block_y + ty) * reg_dim;
    int j = (bx * block_x + tx) * reg_dim;

    float c_sub[reg_dim][reg_dim] = {{0.0f}};

    int num_tiles = (ms + bsize - 1) / bsize;

    for (int t = 0; t < num_tiles; t++) {

        for(int row = 0; row < reg_dim; row++) {
            for(int col = 0; col < reg_dim; col++) {

                int a_row = i + row;
                int a_col = t * bsize + tx * reg_dim + col;
                int as_row = ty * reg_dim + row;
                int as_col = tx * reg_dim + col;

                if (a_row < ms && a_col < ms)
                    as[as_row][as_col] = a[a_row * ms + a_col];
                else
                    as[as_row][as_col] = 0.0f;

                int b_row = t * bsize + ty * reg_dim + row;
                int b_col = j + col;
                int bs_row = ty * reg_dim + row;
                int bs_col = tx * reg_dim + col;

                if (b_row < ms && b_col < ms)
                    bs[bs_row][bs_col] = b[b_row * ms + b_col];
                else
                    bs[bs_row][bs_col] = 0.0f;
            }
        }
        __syncthreads();

        for (int k = 0; k < bsize; k++) {
            float a_reg[reg_dim];
            float b_reg[reg_dim];

            for (int m = 0; m < reg_dim; m++)
                a_reg[m] = as[ty * reg_dim + m][k];

            for (int n = 0; n < reg_dim; n++)
                b_reg[n] = bs[k][tx * reg_dim + n];

            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    c_sub[m][n] += a_reg[m] * b_reg[n];
                }
            }
        }
        __syncthreads();
    }

    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            if ( (i + m) < ms && (j + n) < ms)
                c[(i + m) * ms + (j + n)] = c_sub[m][n];
        }
    }
}

// ===================================================================
// KERNEL 5: Register Tiled + Coalesced Load (from co-registered.cu)
// ===================================================================
__global__ void kernel_reg_coalesced(const float *a, const float *b, float *c) {

    __shared__ float as[bsize][bsize];
    __shared__ float bs[bsize][bsize];

    int ty = threadIdx.y;
    int tx = threadIdx.x;
    int tid = ty * block_x + tx;
    int ib = blockIdx.y * bsize;
    int jb = blockIdx.x * bsize;

    float csub[reg_dim][reg_dim] = {{0.0f}};
    int num_tiles = ms / bsize;

    for (int t = 0; t < num_tiles; t++) {

        int ga_col = t * bsize;
        int gb_row = t * bsize;

        for (int k = 0; k < 16; ++k) {
            int idx = tid + k * 64;
            int sr = idx / 32;
            int sc = idx % 32;
            if((ib + sr) < ms && (ga_col + sc) < ms)
                as[sr][sc] = a[(ib + sr) * ms + (ga_col + sc)];
            else
                as[sr][sc] = 0.0f;

            if((gb_row + sr) < ms && (jb + sc) < ms)
                bs[sr][sc] = b[(gb_row + sr) * ms + (jb + sc)];
            else
                bs[sr][sc] = 0.0f;
        }
        __syncthreads();

        for (int k = 0; k < bsize; k++) {
            float areg[reg_dim];
            float breg[reg_dim];
            for (int m = 0; m < reg_dim; m++) areg[m] = as[ty * reg_dim + m][k];
            for (int n = 0; n < reg_dim; n++) breg[n] = bs[k][tx * reg_dim + n];
            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    csub[m][n] += areg[m] * breg[n];
                }
            }
        }
        __syncthreads();
    }

    __shared__ float cs[bsize][bsize];
    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            cs[ty * reg_dim + m][tx * reg_dim + n] = csub[m][n];
        }
    }
    __syncthreads();

    for (int k = 0; k < 16; ++k) {
        int idx = tid + k * 64;
        int sr = idx / 32;
        int sc = idx % 32;
        if((ib + sr) < ms && (jb + sc) < ms)
            c[(ib + sr) * ms + (jb + sc)] = cs[sr][sc];
    }
}

// ===================================================================
// KERNEL 6: Register Tiled + Coalesced + Double Buffered
// (from not_chosen.cu)
// ===================================================================
__global__ void kernel_reg_double_buffered(const float *a, const float *b, float *c) {

    __shared__ float as[2][bsize][bsize];
    __shared__ float bs[2][bsize][bsize];

    int ty = threadIdx.y;
    int tx = threadIdx.x;
    int tid = ty * block_x + tx;
    int ib = blockIdx.y * bsize;
    int jb = blockIdx.x * bsize;

    float csub[reg_dim][reg_dim] = {{0.0f}};
    int num_tiles = ms / bsize;

    int ga_col = 0 * bsize;
    int gb_row = 0 * bsize;

    for (int k = 0; k < 16; ++k) {
        int idx = tid + k * 64;
        int sr = idx / 32;
        int sc = idx % 32;
        if((ib + sr) < ms && (ga_col + sc) < ms)
            as[0][sr][sc] = a[(ib + sr) * ms + (ga_col + sc)];
        else
            as[0][sr][sc] = 0.0f;

        if((gb_row + sr) < ms && (jb + sc) < ms)
            bs[0][sr][sc] = b[(gb_row + sr) * ms + (jb + sc)];
        else
            bs[0][sr][sc] = 0.0f;
    }
    __syncthreads();

    for (int t = 1; t < num_tiles; t++) {

        int comp_buf = (t - 1) % 2;
        int load_buf = t % 2;

        ga_col = t * bsize;
        gb_row = t * bsize;
        for (int k = 0; k < 16; ++k) {
            int idx = tid + k * 64;
            int sr = idx / 32;
            int sc = idx % 32;
            if((ib + sr) < ms && (ga_col + sc) < ms)
                as[load_buf][sr][sc] = a[(ib + sr) * ms + (ga_col + sc)];
            else
                as[load_buf][sr][sc] = 0.0f;

            if((gb_row + sr) < ms && (jb + sc) < ms)
                bs[load_buf][sr][sc] = b[(gb_row + sr) * ms + (jb + sc)];
            else
                bs[load_buf][sr][sc] = 0.0f;
        }

        for (int k = 0; k < bsize; k++) {
            float areg[reg_dim];
            float breg[reg_dim];
            for (int m = 0; m < reg_dim; m++) areg[m] = as[comp_buf][ty * reg_dim + m][k];
            for (int n = 0; n < reg_dim; n++) breg[n] = bs[comp_buf][k][tx * reg_dim + n];
            for (int m = 0; m < reg_dim; m++) {
                for (int n = 0; n < reg_dim; n++) {
                    csub[m][n] += areg[m] * breg[n];
                }
            }
        }
        __syncthreads();
    }

    int last_buf = (num_tiles - 1) % 2;
    for (int k = 0; k < bsize; k++) {
        float areg[reg_dim];
        float breg[reg_dim];
        for (int m = 0; m < reg_dim; m++) areg[m] = as[last_buf][ty * reg_dim + m][k];
        for (int n = 0; n < reg_dim; n++) breg[n] = bs[last_buf][k][tx * reg_dim + n];
        for (int m = 0; m < reg_dim; m++) {
            for (int n = 0; n < reg_dim; n++) {
                csub[m][n] += areg[m] * breg[n];
            }
        }
    }

    __shared__ float cs[bsize][bsize];
    for(int m = 0; m < reg_dim; m++) {
        for(int n = 0; n < reg_dim; n++) {
            cs[ty * reg_dim + m][tx * reg_dim + n] = csub[m][n];
        }
    }
    __syncthreads();

    for (int k = 0; k < 16; ++k) {
        int idx = tid + k * 64;
        int sr = idx / 32;
        int sc = idx % 32;
        if((ib + sr) < ms && (jb + sc) < ms)
            c[(ib + sr) * ms + (jb + sc)] = cs[sr][sc];
    }
}


// ===================================================================
// HELPER: Matrix Initialization
// ===================================================================
void init_matrix(float *m) {
    for (int i = 0; i < ms * ms; ++i)
        m[i] = (float)rand() / RAND_MAX;
}

// ===================================================================
// MAIN: Benchmarking
// ===================================================================
int main() {
    srand((unsigned int)time(NULL));
    size_t bytes = (size_t)ms * ms * sizeof(float);

    // --- 1. Allocate Host Memory ---
    float *h_a = (float*)malloc(bytes);
    float *h_b = (float*)malloc(bytes);
    float *h_c = (float*)malloc(bytes);

    init_matrix(h_a);
    init_matrix(h_b);

    // --- 2. Allocate Device Memory ---
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
    cudaMemcpy(d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, bytes, cudaMemcpyHostToDevice);

    // --- 3. Setup Benchmarking Tools ---
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cublasHandle_t handle;
    cublasCreate(&handle);
    const float alpha = 1.0f;
    const float beta = 0.0f;

    double gflops_base = (2.0 * (double)ms * (double)ms * (double)ms) / 1e9;
    double cublas_gflops = 0.0;

    float total_ms = 0.0f;
    float ms_kernel = 0.0f;
    double avg_ms = 0.0;
    double gflops = 0.0;
    double percent_of_cublas = 0.0;

    // --- Arrays to store results for the table ---
    const int num_kernels = 7;
    const char* kernel_names[num_kernels] = {
        "1. cuBLAS (Baseline)",
        "2. Naive",
        "3. Tiled",
        "4. Tiled + Double Buffered",
        "5. Register Tiled (Uncoalesced)",
        "6. Register Tiled (Coalesced)",
        "7. Reg. Tiled (Coal + Dbl. Buf)"
    };
    double times[num_kernels];
    double gflops_results[num_kernels];
    double percentages[num_kernels];


    printf("Starting benchmarks for %dx%d matrix multiplication...\n", ms, ms);
    printf("Averaging over %d runs after %d warmup runs.\n\n", benchmark_runs, warmup_runs);

    // --- 4. Define Grid/Block Dims ---
    dim3 block_std(bsize, bsize);
    dim3 grid_std((ms + bsize - 1) / bsize, (ms + bsize - 1) / bsize);

    dim3 block_reg(block_x, block_y);
    dim3 grid_reg_tiled(ms / (block_x * reg_dim), ms / (block_y * reg_dim));
    dim3 grid_reg_coalesced(ms / bsize, ms / bsize);
    dim3 grid_tiled_db(ms / bsize, ms / bsize);

    // ===================================
    // BENCHMARK 1: cuBLAS (Baseline)
    // ===================================
    printf("Benchmarking %s...", kernel_names[0]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, ms, ms, ms, &alpha, d_a, ms, d_b, ms, &beta, d_c, ms);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, ms, ms, ms, &alpha, d_a, ms, d_b, ms, &beta, d_c, ms);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    cublas_gflops = gflops_base / (avg_ms / 1e3);

    times[0] = avg_ms;
    gflops_results[0] = cublas_gflops;
    percentages[0] = 100.0;
    printf(" Done.\n");

    // ===================================
    // BENCHMARK 2: Naive
    // ===================================
    printf("Benchmarking %s...", kernel_names[1]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        kernel_naive<<<grid_std, block_std>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        kernel_naive<<<grid_std, block_std>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    gflops = gflops_base / (avg_ms / 1e3);
    percent_of_cublas = (gflops / cublas_gflops) * 100.0;

    times[1] = avg_ms;
    gflops_results[1] = gflops;
    percentages[1] = percent_of_cublas;
    printf(" Done.\n");

    // ===================================
    // BENCHMARK 3: Tiled
    // ===================================
    printf("Benchmarking %s...", kernel_names[2]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        kernel_tiled<<<grid_std, block_std>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        kernel_tiled<<<grid_std, block_std>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    gflops = gflops_base / (avg_ms / 1e3);
    percent_of_cublas = (gflops / cublas_gflops) * 100.0;

    times[2] = avg_ms;
    gflops_results[2] = gflops;
    percentages[2] = percent_of_cublas;
    printf(" Done.\n");

    // ===================================
    // BENCHMARK 4: Tiled + Double Buffered
    // ===================================
    printf("Benchmarking %s...", kernel_names[3]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        kernel_tiled_double_buffered<<<grid_tiled_db, block_std>>>(d_a, d_b, d_c, ms);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        kernel_tiled_double_buffered<<<grid_tiled_db, block_std>>>(d_a, d_b, d_c, ms);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    gflops = gflops_base / (avg_ms / 1e3);
    percent_of_cublas = (gflops / cublas_gflops) * 100.0;

    times[3] = avg_ms;
    gflops_results[3] = gflops;
    percentages[3] = percent_of_cublas;
    printf(" Done.\n");

    // ===================================
    // BENCHMARK 5: Register Tiled (Uncoalesced)
    // ===================================
    printf("Benchmarking %s...", kernel_names[4]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        kernel_reg_tiled<<<grid_reg_tiled, block_reg>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        kernel_reg_tiled<<<grid_reg_tiled, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    gflops = gflops_base / (avg_ms / 1e3);
    percent_of_cublas = (gflops / cublas_gflops) * 100.0;

    times[4] = avg_ms;
    gflops_results[4] = gflops;
    percentages[4] = percent_of_cublas;
    printf(" Done.\n");

    // ===================================
    // BENCHMARK 6: Register Tiled (Coalesced)
    // ===================================
    printf("Benchmarking %s...", kernel_names[5]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        kernel_reg_coalesced<<<grid_reg_coalesced, block_reg>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        kernel_reg_coalesced<<<grid_reg_coalesced, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    gflops = gflops_base / (avg_ms / 1e3);
    percent_of_cublas = (gflops / cublas_gflops) * 100.0;

    times[5] = avg_ms;
    gflops_results[5] = gflops;
    percentages[5] = percent_of_cublas;
    printf(" Done.\n");

    // ===================================
    // BENCHMARK 7: Register Tiled (Coalesced + Dbl. Buffered)
    // ===================================
    printf("Benchmarking %s...", kernel_names[6]);
    total_ms = 0.0f;
    for (int i = 0; i < warmup_runs; ++i)
        kernel_reg_double_buffered<<<grid_reg_coalesced, block_reg>>>(d_a, d_b, d_c);
    cudaDeviceSynchronize();

    for (int i = 0; i < benchmark_runs; ++i) {
        cudaMemset(d_c, 0, bytes);
        cudaEventRecord(start);
        kernel_reg_double_buffered<<<grid_reg_coalesced, block_reg>>>(d_a, d_b, d_c);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_kernel, start, stop);
        total_ms += ms_kernel;
    }
    avg_ms = (double)total_ms / benchmark_runs;
    gflops = gflops_base / (avg_ms / 1e3);
    percent_of_cublas = (gflops / cublas_gflops) * 100.0;

    times[6] = avg_ms;
    gflops_results[6] = gflops;
    percentages[6] = percent_of_cublas;
    printf(" Done.\n\n");


    // ===================================
    // FINAL: Print Comparison Matrix
    // ===================================
    printf("+------------------------------------+-------------+-------------+---------------+\n");
    printf("| Kernel                             | Avg. Time   | Performance | % of cuBLAS   |\n");
    printf("|                                    | (ms)        | (GFLOPS)    | Performance   |\n");
    printf("+------------------------------------+-------------+-------------+---------------+\n");

    for (int i = 0; i < num_kernels; ++i) {
        printf("| %-34s | %11.3f | %11.3f | %13.2f%% |\n",
               kernel_names[i],
               times[i],
               gflops_results[i],
               percentages[i]);
        if (i == 0) {
            printf("+------------------------------------+-------------+-------------+---------------+\n");
        }
    }
    printf("+------------------------------------+-------------+-------------+---------------+\n");


    // --- 5. Cleanup ---
    cublasDestroy(handle);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing bm.cu


In [20]:
!nvcc -arch=sm_75 -o  bm bm.cu -lcublas
!./bm

bm.cu(647): warning #224-D: the format string requires additional arguments
      printf("| Kernel                             | Avg. Time   | Performance | % of cuBLAS   |\n");
                                                                                                   ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

bm.cu: In function ‘int main()’:
bm.cu:647:8: warning: ' ' flag used with ‘%o’ gnu_printf format []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
  647 |     printf("| Kernel                             | Avg. Time   | Performance | % of cuBLAS   |\n");
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
bm.cu:647:8: warning: format ‘%o’ expects a matching ‘unsigned int’ argument []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
Starting benchmarks for 1024x1024 matrix multiplication...
Averaging 